# Quentization
- ref: https://colab.research.google.com/drive/1suUbtdS64f_ZvbEcWehIheg9G-88_EQE?hl=en

## AbsMax Quantization

In [1]:
import numpy as np

In [2]:
def quantize_fp16_to_int8(fp16_weights):
    """
    quantize fp16 to int8
    e.g. [1.2, 0.3, -0.1, -0.8]

    change [-0.8, 1.2] to [-128, 127]
    先透過正規化將其 Normalize 到 [-128, 127] 的範圍

    Step1: 先減去最小值: -0.8
    [2.0, 1.1, 0.7, 0.0]

    Step2: 再除以最大值: 2.0
    [1.0, 0.55, 0.35, 0.0]

    Step3: 最後縮放到 [-128, 127] INT8的範圍: x * 255 - 128
    [127, 12, -39, -128]

    以上為 FP16 到 INT8 的量化 (Quantize) 動作
    """
    # 步驟 1: 減去最小值
    min_val = np.min(fp16_weights)
    normalized_weights = fp16_weights - min_val
    
    # 步驟 2: 除以最大值
    max_val = np.max(normalized_weights)
    normalized_weights /= max_val
    
    # 步驟 3: 縮放到 INT8 範圍 [-128, 127]
    int8_weights = np.round(normalized_weights * 255 - 128).astype(np.int8)
    
    return int8_weights, min_val, max_val

def dequantize_int8_to_fp16(int8_weights, min_val, max_val):
    """
    在存放這組數列時，"最小值" 與 "間距" 也要一起被存下來，通常會是原本的資料型態。
    除了精準度的考量以外，也要考慮 "極值的問題" ，避免間距或最小值因為過大過小而發生溢位的問題。

    接著在運算時，通常會將 "數列反向量化 (Dequantize) 回來"，也就是按照量化的流程反向計算回去
    (x + 128) / 255 * 2.0 + (-0.8)

    雖然四個數值省下了 8 * 4 = 32 個位元，但是因為額外存了兩個 FP16 的資訊，所以兩者就抵銷了
    """
    # 按照量化的流程反向計算回去
    fp16_weights = (int8_weights + 128) / 255 * max_val + min_val
    return fp16_weights.astype(np.float16)

# 給定的 FP16 數據
fp16_weights = np.array([1.2, 0.3, -0.1, -0.8], dtype=np.float16)

# 進行量化
int8_weights, min_val, max_val = quantize_fp16_to_int8(fp16_weights)

# 進行反向量化
reconstructed_fp16 = dequantize_int8_to_fp16(int8_weights, min_val, max_val)

int8_weights, reconstructed_fp16

(array([ 127,   12,  -39, -128], dtype=int8),
 array([ 1.2    ,  0.2983 , -0.10175, -0.8    ], dtype=float16))

In [3]:
# AbsMax Quantization
def quantize_fp16_to_int8(fp16_weights):
    """
    quantize fp16 to int8
    e.g. [1.2, 0.3, -0.1, -0.8]

    Step1: 先對數列取絕對值再做量化
    [1.2, 0.3, 0.1, 0.8]

    Step2: 找到權重的絕對最大值並除之正規化到 [-1.0, 1.0] 之間: 1.2
    [1.0, 0.25, 0.083, 0.67]

    Step3: 最後縮放到 [-128, 127] INT8的範圍: x * 127
    [127, 32, 10, 81]

    Step4: 取得 "量化常數(Quantization Constant)"，透過量化常數與原本的數列相乘並四捨五入
    (x / 1.2) * 127
    = x * (1 / 1.2) * 127  # 改寫為乘法
    = x * 127 * (1 / 1.2)  # 根據交換律
    = x * (127 / 1.2)      # 根據結合律
    = x * 105.83           # 量化常數(Quantization Constant)

    這也是這個量化方法裡面唯一一個額外資訊
    [1.2, 0.3, -0.1, -0.8] * 105.83
    = [127, 32, -11, -85]
    """
    # 找到權重的絕對最大值
    abs_max_val = np.max(np.abs(fp16_weights))

    # 量化常數 Quantization Constant
    quant_const = 127 / abs_max_val

    # 將 FP16 權重轉換為 Int8
    weights = fp16_weights * quant_const
    int8_weights = np.round(weights).astype(np.int8)

    return int8_weights, quant_const


def dequantize_int8_to_fp16(int8_weights, quant_const):
    """
    反向量化也只要除以量化常數即可
    """
    # 將 Int8 權重轉換回 FP16
    return int8_weights.astype(np.float16) / quant_const

# 給定的 FP16 數據
fp16_weights = np.array([1.2, 0.3, -0.1, -0.8], dtype=np.float16)

# 進行量化
int8_weights, quant_const = quantize_fp16_to_int8(fp16_weights)

# 進行反向量化
reconstructed_fp16 = dequantize_int8_to_fp16(int8_weights, quant_const)

int8_weights, reconstructed_fp16

(array([127,  32, -11, -85], dtype=int8),
 array([ 1.2    ,  0.3025 , -0.10394, -0.803  ], dtype=float16))

## Simple Model

In [5]:
# 初始化隨機數生成器的種子，以確保結果可重現
np.random.seed(2135)

# 設定輸入樣本數量、隱藏層大小與輸出類別數量
inn_size = 12
hid_size = 512
out_size = 4096

# 產生模型輸入、隱藏層權重與分類器權重
x = np.random.randn(inn_size, hid_size).astype(np.float16)
w_fp16 = np.random.randn(hid_size, hid_size).astype(np.float16)
clf = np.random.randn(hid_size, out_size).astype(np.float16)

hid = np.matmul(x, w_fp16)  # 計算隱藏層輸出
out = np.matmul(hid, clf)  # 計算分類器輸出
y = np.argmax(out, -1)  # 實際類別預測
y

array([3956,  874,  201,  109,  843,   46, 1419, 1058,  865, 2894, 2059,
       1386], dtype=int64)

In [6]:
# 進行 Int8 Quantization 並再次計算一次
w_int8, sf = quantize_fp16_to_int8(w_fp16)
rw_fp16 = dequantize_int8_to_fp16(w_int8, sf)
hid = np.matmul(x, rw_fp16)
out = np.matmul(hid, clf)
y_int8 = np.argmax(out, -1)

# 比較 Int8 量化的預測結果與原始輸出的差異
errors_int8 = np.sum(np.not_equal(y, y_int8))

print(f"FP16 Prediction: {y}")
print(f"Int8 Prediction: {y_int8}")
print(f"Int8 Error: {errors_int8}, Results: {y == y_int8}")

FP16 Prediction: [3956  874  201  109  843   46 1419 1058  865 2894 2059 1386]
Int8 Prediction: [3956  874  201  109  843   46 1419 1058 1495 2894 2059 1386]
Int8 Error: 1, Results: [ True  True  True  True  True  True  True  True False  True  True  True]
